# **Scrapping Algorithm**

In [1]:
from bs4 import BeautifulSoup  # to parse html
from datetime import datetime
import os
import pandas as pd

# import requests  # to send http requests
from pip._vendor import requests
from time import (
    sleep,
)  # function that allows you to pause the program. helpful to spae out requests
from tqdm import tqdm

c:\Users\Luismi\anaconda3\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [22]:
# Define our url: https://www.failory.com/startups/germany

html_text = requests.get('https://www.failory.com/startups/germany', timeout=15)

In [70]:
# Getting the info from the beautifulSoup
soup = BeautifulSoup(html_text.content, 'html.parser')
startups = soup.find_all('h3')
lists = soup.find_all('ul')
startups = [str(i) for i in startups]
lists = [str(i) for i in lists]
lists[0]

'<ul role="list"><li>City: Munich</li><li>Started in: 2011</li><li>Founders: Alexander Rinke, Bastian Nominacher, Martin Klenk</li><li>Industries: Analytics, Business Intelligence, SaaS, Software</li><li>Number of employees: 1001-5000</li><li>Funding amount: $1,367,500,000</li><li>Number of funding rounds: 4</li><li>Number of investors: 9</li></ul>'

In [71]:
startups = [i.strip("<h3>/)1234567890 ") for i in startups]
startups

['Celonis',
 'TradeRepublic',
 'Ada Healt',
 'Delivery Hero',
 'uberall',
 'TIER Mobility',
 'Foodpanda',
 'Scalable Capital',
 'Trade Republic',
 'FlixBus',
 'wefox',
 'HelloFres',
 'SoundCloud',
 'Mambu',
 'Lilium',
 'Flink Food',
 'Razor Group',
 'Forto',
 'Sanity Group',
 'WunderMobility',
 'OroraTec',
 'YFood',
 'Habyt',
 'OneFootball',
 'parcelLab',
 'ELEMENT Insurance',
 'Contentful',
 'Spryker Systems',
 'ATAI Life Sciences',
 'Adjust',
 'TIER',
 'CoachHub',
 'Vivid Money',
 'DHL',
 'Infarm',
 'Solarisbank',
 'GetYourGuide',
 'TWAICE',
 'Isar Aerospace',
 'undred',
 'HomeToGo',
 'Personio',
 'QUIN',
 'Volocopter',
 'Holidu',
 'Avi Medical',
 'Grover',
 'Raccoon.World',
 'Cognigy GmbH',
 'commercetools',
 'Lumenaza',
 'Enpal',
 'Zeotap',
 'airfocus',
 'Morressier',
 'Bunc',
 'Alasco',
 'Zalando',
 'SimScale',
 'Choco',
 'AUTO1 Group',
 'Billie',
 'Finoa',
 'sevDesk',
 'Signavio',
 'Babbel',
 'Silexica',
 'Raisin',
 'IDnow',
 'Pricefx',
 'BRYTER',
 'Arabesque S-Ray',
 'Camunda',


In [76]:
import re

CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});') 

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

lists = []